In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import os
ROOT = '/content/drive/MyDrive/Midfluencer'
INPUT = os.path.join(ROOT, "images")
OUTPUT = os.path.join(ROOT, "output")

In [4]:
MODEL = "dreamlike-art/dreamlike-photoreal-2.0"

In [5]:
!pip install -U autotrain-advanced

In [1]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [6]:
!autotrain dreambooth --help

usage: autotrain <command> [<args>] dreambooth [-h] --model MODEL [--revision REVISION]
                                               [--tokenizer TOKENIZER] --image-path IMAGE_PATH
                                               [--class-image-path CLASS_IMAGE_PATH] --prompt
                                               PROMPT [--class-prompt CLASS_PROMPT]
                                               [--num-class-images NUM_CLASS_IMAGES]
                                               [--class-labels-conditioning CLASS_LABELS_CONDITIONING]
                                               [--prior-preservation]
                                               [--prior-loss-weight PRIOR_LOSS_WEIGHT]
                                               --project-name PROJECT_NAME [--seed SEED]
                                               --resolution RESOLUTION [--center-crop]
                                               [--train-text-encoder] [--batch-size BATCH_SIZE]
                      

In [ ]:
!autotrain dreambooth --class-prompt "photo of a person" \
--model dreamlike-art/dreamlike-photoreal-2.0 \
--image-path /content/drive/MyDrive/Midfluencer/images \
--prompt "photo of LaurAI3 person" \
--resolution 1024 \
--batch-size 1 \
--num-steps 1000 \
--fp \
--gradient-accumulation 4 \
--lr 1e-4 \
--project-name /content/drive/MyDrive/Midfluencer/output/midfluencer_v7


> INFO    Namespace(version=False, model='dreamlike-art/dreamlike-photoreal-2.0', revision=None, tokenizer=None, image_path='/content/drive/MyDrive/Midfluencer/images', class_image_path=None, prompt='photo of LaurAI3 person', class_prompt='photo of a person', num_class_images=100, class_labels_conditioning=None, prior_preservation=None, prior_loss_weight=1.0, project_name='/content/drive/MyDrive/Midfluencer/output/midfluencer_v7', seed=42, resolution=1024, center_crop=None, train_text_encoder=None, batch_size=1, sample_batch_size=4, epochs=1, num_steps=1000, checkpointing_steps=100000, resume_from_checkpoint=None, gradient_accumulation=4, gradient_checkpointing=None, lr=0.0001, scale_lr=None, scheduler='constant', warmup_steps=0, num_cycles=1, lr_power=1.0, dataloader_num_workers=0, use_8bit_adam=None, adam_beta1=0.9, adam_beta2=0.999, adam_weight_decay=0.01, adam_epsilon=1e-08, max_grad_norm=1.0, allow_tf32=None, prior_generation_precision=None, local_rank=-1, xformers=None, pre_compu

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

version = "_v7"
if not os.path.exists(os.path.join(OUTPUT, f"raw{version}")):
  os.mkdir(os.path.join(OUTPUT, f"raw{version}"))
if not os.path.exists(os.path.join(OUTPUT, f"processed{version}")):
  os.mkdir(os.path.join(OUTPUT, f"processed{version}"))

pipe = DiffusionPipeline.from_pretrained(
    MODEL,
    torch_dtype=torch.float16
)
pipe.to("cuda")
pipe.load_lora_weights(os.path.join(OUTPUT, "midfluencer"+version), weight_name="pytorch_lora_weights.safetensors")

prompt = "12K, HRD, film grain, 100 ASA, The image features the woman LaurAI3 walking down a city street, wearing a black tank top and blue jeans. She is also carrying a handbag. The street is busy with several other people walking around, and there are multiple cars parked or driving along the street. Traffic lights can be seen at various points, indicating that the street is well-regulated for pedestrian and vehicular traffic. The woman appears to be confidently strolling down the sidewalk, enjoying her time in the city. Hyperrealism 1. 5, masterpiece 1. 0, Ultra HD"

for seed in range(10):
    generator = torch.Generator("cuda").manual_seed(seed)
    image_pipe = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image_pipe.images[0]
    image.save(os.path.join(OUTPUT, f"raw{version}", f"{seed}.png"))